# **Chapter 11: State Management Architecture**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Distinguish between ephemeral (UI) state and application (business) state
- Implement setState effectively and understand its limitations for scalability
- Utilize ValueNotifier and ChangeNotifier for simple state management
- Implement the Provider pattern for dependency injection and state propagation
- Apply Riverpod for compile-safe, testable state management without BuildContext dependencies
- Architect applications using BLoC pattern with proper event/state separation
- Choose appropriate state management solutions based on application complexity

---

## **Prerequisites**

- Completed Chapter 10: Input, Forms, and Validation
- Solid understanding of StatefulWidget and lifecycle methods
- Familiarity with InheritedWidget and context
- Understanding of Streams and Futures from Chapter 6
- Experience with object-oriented design patterns

---

## **11.1 State Management Fundamentals**

State management is one of the most critical architectural decisions in Flutter. Understanding the distinction between different types of state and when to use each solution is essential for building maintainable applications.

### **Ephemeral vs. App State**

```dart
import 'package:flutter/material.dart';

void main() {
  runApp(StateTypesDemo());
}

class StateTypesDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      home: StateClassificationPage(),
    );
  }
}

// Example 1: Ephemeral State (UI State)
// State that is local to a single widget and doesn't need to be shared
class CounterButton extends StatefulWidget {
  @override
  _CounterButtonState createState() => _CounterButtonState();
}

class _CounterButtonState extends State<CounterButton> {
  // This is ephemeral state - only this widget cares about this value
  int _counter = 0;

  void _increment() {
    setState(() {
      _counter++;
    });
  }

  @override
  Widget build(BuildContext context) {
    return ElevatedButton(
      onPressed: _increment,
      child: Text('Clicked $_counter times'),
    );
  }
}

// Example 2: App State (Shared State)
// State that needs to be shared across multiple widgets or screens
class ShoppingCart extends ChangeNotifier {
  // This is app state - multiple widgets need to know about cart contents
  final List<String> _items = [];

  List<String> get items => List.unmodifiable(_items);

  void addItem(String item) {
    _items.add(item);
    notifyListeners(); // Notify all listeners that state changed
  }

  void removeItem(String item) {
    _items.remove(item);
    notifyListeners();
  }

  int get itemCount => _items.length;
}

// Example 3: Page State (Intermediate)
// State that is shared within a page but not globally
class CheckoutPage extends StatefulWidget {
  @override
  _CheckoutPageState createState() => _CheckoutPageState();
}

class _CheckoutPageState extends State<CheckoutPage> {
  // Page-level state: Payment method selection affects multiple form sections
  // but doesn't need to persist after leaving this page
  String _selectedPaymentMethod = 'credit_card';
  bool _isProcessing = false;

  void _selectPaymentMethod(String method) {
    setState(() {
      _selectedPaymentMethod = method;
    });
  }

  @override
  Widget build(BuildContext context) {
    return Column(
      children: [
        PaymentMethodSelector(
          selectedMethod: _selectedPaymentMethod,
          onMethodSelected: _selectPaymentMethod,
        ),
        PaymentForm(
          paymentMethod: _selectedPaymentMethod,
          isProcessing: _isProcessing,
        ),
      ],
    );
  }
}

class StateClassificationPage extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('State Types')),
      body: Padding(
        padding: EdgeInsets.all(16),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            Text(
              'Ephemeral State (setState)',
              style: TextStyle(fontSize: 20, fontWeight: FontWeight.bold),
            ),
            Text('Local to a single widget, temporary UI state'),
            SizedBox(height: 8),
            CounterButton(),
            
            SizedBox(height: 32),
            
            Text(
              'App State (Shared)',
              style: TextStyle(fontSize: 20, fontWeight: FontWeight.bold),
            ),
            Text('Shared across the app, persistent business logic'),
            SizedBox(height: 8),
            // This would be provided higher in the tree
            CartSummary(),
          ],
        ),
      ),
    );
  }
}

class CartSummary extends StatelessWidget {
  // In a real app, this would use Provider or similar to access ShoppingCart
  @override
  Widget build(BuildContext context) {
    return Card(
      child: ListTile(
        leading: Icon(Icons.shopping_cart),
        title: Text('Cart Items'),
        trailing: Chip(label: Text('3 items')), // Would be dynamic
      ),
    );
  }
}

// Placeholder widgets for the example
class PaymentMethodSelector extends StatelessWidget {
  final String selectedMethod;
  final Function(String) onMethodSelected;

  const PaymentMethodSelector({
    Key? key,
    required this.selectedMethod,
    required this.onMethodSelected,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Container(); // Implementation details omitted
  }
}

class PaymentForm extends StatelessWidget {
  final String paymentMethod;
  final bool isProcessing;

  const PaymentForm({
    Key? key,
    required this.paymentMethod,
    required this.isProcessing,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Container(); // Implementation details omitted
  }
}
```

**Explanation:**

- **Ephemeral State**: Also called UI state or local state. This is state that is temporary and local to a specific widget. Examples include:
  - Current page in a PageView
  - Current selected tab in a TabBar
  - Animation state
  - Text field focus state
  
  Use `setState()` for ephemeral state. It's simple, requires no boilerplate, and keeps the state close to where it's used.

- **App State**: Also called shared state or business logic. This is state that needs to be shared across multiple parts of your app. Examples include:
  - User authentication status
  - Shopping cart contents
  - Theme preferences
  - Notification count
  
  Use dedicated state management solutions (Provider, Riverpod, BLoC, etc.) for app state. These provide testability, separation of concerns, and scalability.

- **Page State**: Intermediate state that is shared among widgets within a specific page/screen but doesn't need to persist beyond that screen. This can be managed by the page's StatefulWidget and passed down via constructor parameters, or via Provider scoped to that page.

- **Decision criteria**:
  - If only one widget needs the state → setState
  - If multiple widgets in one screen need it → Page-level state (StatefulWidget or Provider)
  - If multiple screens need it → App-level state management

### **The Limitations of setState**

```dart
import 'package:flutter/material.dart';

// This example demonstrates why setState doesn't scale well

void main() {
  runApp(PropDrillingDemo());
}

class PropDrillingDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      home: PropDrillingProblem(),
    );
  }
}

// Problem: Prop Drilling
// When using only setState, we have to pass state and callbacks
// through every widget in the tree, even if intermediate widgets don't need them

class PropDrillingProblem extends StatefulWidget {
  @override
  _PropDrillingProblemState createState() => _PropDrillingProblemState();
}

class _PropDrillingProblemState extends State<PropDrillingProblem> {
  String _userName = 'John Doe';
  int _notificationCount = 5;

  void _updateUserName(String name) {
    setState(() {
      _userName = name;
    });
  }

  void _incrementNotifications() {
    setState(() {
      _notificationCount++;
    });
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Prop Drilling Problem')),
      body: LevelOne(
        userName: _userName,
        notificationCount: _notificationCount,
        onUpdateUserName: _updateUserName,
        onIncrementNotifications: _incrementNotifications,
      ),
    );
  }
}

// Level One doesn't use these values, but must pass them through
class LevelOne extends StatelessWidget {
  final String userName;
  final int notificationCount;
  final Function(String) onUpdateUserName;
  final VoidCallback onIncrementNotifications;

  const LevelOne({
    Key? key,
    required this.userName,
    required this.notificationCount,
    required this.onUpdateUserName,
    required this.onIncrementNotifications,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Container(
      padding: EdgeInsets.all(16),
      child: LevelTwo(
        userName: userName,
        notificationCount: notificationCount,
        onUpdateUserName: onUpdateUserName,
        onIncrementNotifications: onIncrementNotifications,
      ),
    );
  }
}

// Level Two also doesn't use these values
class LevelTwo extends StatelessWidget {
  final String userName;
  final int notificationCount;
  final Function(String) onUpdateUserName;
  final VoidCallback onIncrementNotifications;

  const LevelTwo({
    Key? key,
    required this.userName,
    required this.notificationCount,
    required this.onUpdateUserName,
    required this.onIncrementNotifications,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Card(
      child: Padding(
        padding: EdgeInsets.all(16),
        child: LevelThree(
          userName: userName,
          notificationCount: notificationCount,
          onUpdateUserName: onUpdateUserName,
          onIncrementNotifications: onIncrementNotifications,
        ),
      ),
    );
  }
}

// Level Three finally uses the values
class LevelThree extends StatelessWidget {
  final String userName;
  final int notificationCount;
  final Function(String) onUpdateUserName;
  final VoidCallback onIncrementNotifications;

  const LevelThree({
    Key? key,
    required this.userName,
    required this.notificationCount,
    required this.onUpdateUserName,
    required this.onIncrementNotifications,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Column(
      children: [
        Text('User: $userName'),
        Text('Notifications: $notificationCount'),
        TextField(
          decoration: InputDecoration(labelText: 'Update Name'),
          onSubmitted: onUpdateUserName,
        ),
        ElevatedButton(
          onPressed: onIncrementNotifications,
          child: Text('Add Notification'),
        ),
      ],
    );
  }
}

// Problem 2: Rebuild Performance
// With setState, the entire widget tree rebuilds even if only one small part changed

class RebuildProblem extends StatefulWidget {
  @override
  _RebuildProblemState createState() => _RebuildProblemState();
}

class _RebuildProblemState extends State<RebuildProblem> {
  int _counter1 = 0;
  int _counter2 = 0;

  @override
  Widget build(BuildContext context) {
    print('Building Parent - expensive operation!');
    
    return Scaffold(
      appBar: AppBar(title: Text('Rebuild Problem')),
      body: Column(
        children: [
          // When we call setState for counter1, both Counter1Display 
          // AND Counter2Display rebuild, even though counter2 didn't change
          Counter1Display(
            count: _counter1,
            onIncrement: () => setState(() => _counter1++),
          ),
          Counter2Display(
            count: _counter2,
            onIncrement: () => setState(() => _counter2++),
          ),
          // ExpensiveWidget rebuilds unnecessarily when either counter changes
          ExpensiveWidget(),
        ],
      ),
    );
  }
}

class Counter1Display extends StatelessWidget {
  final int count;
  final VoidCallback onIncrement;

  const Counter1Display({
    Key? key,
    required this.count,
    required this.onIncrement,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    print('Building Counter 1');
    return Card(
      child: ListTile(
        title: Text('Counter 1: $count'),
        trailing: ElevatedButton(
          onPressed: onIncrement,
          child: Text('+'),
        ),
      ),
    );
  }
}

class Counter2Display extends StatelessWidget {
  final int count;
  final VoidCallback onIncrement;

  const Counter2Display({
    Key? key,
    required this.count,
    required this.onIncrement,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    print('Building Counter 2');
    return Card(
      child: ListTile(
        title: Text('Counter 2: $count'),
        trailing: ElevatedButton(
          onPressed: onIncrement,
          child: Text('+'),
        ),
      ),
    );
  }
}

class ExpensiveWidget extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    print('Building Expensive Widget - this is slow!');
    // Simulating expensive build
    return Container(
      height: 200,
      color: Colors.blue[100],
      child: Center(child: Text('Expensive Widget')),
    );
  }
}
```

**Explanation:**

- **Prop Drilling**: When using setState for app-wide state, you must pass data and callbacks through intermediate widgets that don't need them. This creates:
  - Boilerplate code
  - Tight coupling between unrelated widgets
  - Difficulty refactoring (changing one widget affects many)
  - Reduced code readability

- **Rebuild Performance**: When you call setState() in a StatefulWidget, Flutter marks that widget as dirty and rebuilds it. Since build methods return immutable widget trees, Flutter rebuilds the entire subtree, even if only one small part actually changed.
  - In the example, updating counter1 causes ExpensiveWidget to rebuild unnecessarily
  - This leads to performance issues in large applications
  - Solutions like Provider use InheritedWidget underneath to only rebuild dependents, not the entire tree

- **When setState is insufficient**:
  - State needed by multiple screens
  - Deep widget trees (more than 2-3 levels)
  - Complex state logic (validation, async operations)
  - Need for caching or persistence
  - Testing requirements (business logic separated from UI)

---

## **11.2 ValueNotifier and ChangeNotifier**

Before diving into complex state management packages, Flutter provides built-in solutions that are lightweight and effective for medium complexity: ValueNotifier and ChangeNotifier.

### **ValueNotifier for Simple Values**

```dart
import 'package:flutter/material.dart';

void main() {
  runapp(ValueNotifierDemo());
}

// ValueNotifier is perfect for single primitive values or simple objects
class Counter extends ValueNotifier<int> {
  Counter() : super(0);

  void increment() => value++;
  void decrement() => value--;
  void reset() => value = 0;
}

// For complex objects, create a custom ValueNotifier
class User extends ValueNotifier<UserData> {
  User() : super(UserData(name: '', age: 0));

  void updateName(String name) {
    value = value.copyWith(name: name);
  }

  void updateAge(int age) {
    value = value.copyWith(age: age);
  }
}

class UserData {
  final String name;
  final int age;

  UserData({required this.name, required this.age});

  UserData copyWith({String? name, int? age}) {
    return UserData(
      name: name ?? this.name,
      age: age ?? this.age,
    );
  }
}

class ValueNotifierDemo extends StatelessWidget {
  // Create notifiers at the level where they're needed
  final Counter _counter = Counter();
  final User _user = User();

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('ValueNotifier')),
      body: Padding(
        padding: EdgeInsets.all(16),
        child: Column(
          children: [
            // Listening to ValueNotifier with ValueListenableBuilder
            // This is more efficient than setState because only this widget rebuilds
            ValueListenableBuilder<int>(
              valueListenable: _counter,
              builder: (context, count, child) {
                print('Building counter display: $count');
                return Card(
                  child: ListTile(
                    title: Text('Count: $count'),
                    trailing: Row(
                      mainAxisSize: MainAxisSize.min,
                      children: [
                        IconButton(
                          icon: Icon(Icons.remove),
                          onPressed: _counter.decrement,
                        ),
                        IconButton(
                          icon: Icon(Icons.add),
                          onPressed: _counter.increment,
                        ),
                      ],
                    ),
                  ),
                );
              },
            ),
            
            SizedBox(height: 20),
            
            // Complex object with ValueNotifier
            ValueListenableBuilder<UserData>(
              valueListenable: _user,
              builder: (context, userData, child) {
                print('Building user display');
                return Column(
                  children: [
                    TextField(
                      decoration: InputDecoration(labelText: 'Name'),
                      onChanged: _user.updateName,
                    ),
                    SizedBox(height: 8),
                    TextField(
                      decoration: InputDecoration(labelText: 'Age'),
                      keyboardType: TextInputType.number,
                      onChanged: (value) => _user.updateAge(int.tryParse(value) ?? 0),
                    ),
                    SizedBox(height: 16),
                    Text('Current User: ${userData.name}, ${userData.age}'),
                  ],
                );
              },
            ),
          ],
        ),
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: _counter.reset,
        child: Icon(Icons.refresh),
      ),
    );
  }
}
```

**Explanation:**

- **`ValueNotifier<T>`**: A lightweight observable that holds a single value. It extends `ChangeNotifier` but is specialized for single values.

- **`ValueListenableBuilder`**: A widget that listens to a ValueNotifier and rebuilds only when the value changes. This is much more efficient than setState because:
  - Only the ValueListenableBuilder rebuilds
  - Parent widgets don't rebuild
  - Other parts of the UI remain unaffected

- **Immutability**: When updating complex objects in ValueNotifier, always create new instances (copyWith) rather than mutating existing objects. This ensures proper change detection.

- **Use cases**:
  - Simple counters or toggles
  - Current page index
  - Form field values
  - Animation progress
  - Theme mode

### **ChangeNotifier for Complex State**

```dart
import 'package:flutter/material.dart';

void main() {
  runapp(ChangeNotifierDemo());
}

// ChangeNotifier is the foundation for more complex state management
// It's part of Flutter's foundation library, no external packages needed

class Todo {
  final String id;
  final String title;
  final bool isCompleted;

  Todo({
    required this.id,
    required this.title,
    this.isCompleted = false,
  });

  Todo copyWith({String? id, String? title, bool? isCompleted}) {
    return Todo(
      id: id ?? this.id,
      title: title ?? this.title,
      isCompleted: isCompleted ?? this.isCompleted,
    );
  }
}

class TodoController extends ChangeNotifier {
  // Private state
  final List<Todo> _todos = [];
  String _filter = 'all'; // 'all', 'active', 'completed'

  // Public getters (read-only access)
  List<Todo> get todos {
    switch (_filter) {
      case 'active':
        return _todos.where((t) => !t.isCompleted).toList();
      case 'completed':
        return _todos.where((t) => t.isCompleted).toList();
      default:
        return List.unmodifiable(_todos);
    }
  }

  String get filter => _filter;
  
  int get todoCount => _todos.length;
  int get completedCount => _todos.where((t) => t.isCompleted).length;
  int get activeCount => _todos.where((t) => !t.isCompleted).length;

  // Business logic methods
  void addTodo(String title) {
    if (title.trim().isEmpty) return;
    
    _todos.add(Todo(
      id: DateTime.now().toString(),
      title: title.trim(),
    ));
    notifyListeners(); // Notify all listeners that state changed
  }

  void toggleTodo(String id) {
    final index = _todos.indexWhere((t) => t.id == id);
    if (index != -1) {
      _todos[index] = _todos[index].copyWith(
        isCompleted: !_todos[index].isCompleted,
      );
      notifyListeners();
    }
  }

  void removeTodo(String id) {
    _todos.removeWhere((t) => t.id == id);
    notifyListeners();
  }

  void setFilter(String filter) {
    _filter = filter;
    notifyListeners();
  }

  void clearCompleted() {
    _todos.removeWhere((t) => t.isCompleted);
    notifyListeners();
  }
}

class ChangeNotifierDemo extends StatefulWidget {
  @override
  _ChangeNotifierDemoState createState() => _ChangeNotifierDemoState();
}

class _ChangeNotifierDemoState extends State<ChangeNotifierDemo> {
  // In a real app, this would be provided higher in the tree
  late final TodoController _controller;
  final TextEditingController _textController = TextEditingController();

  @override
  void initState() {
    super.initState();
    _controller = TodoController();
  }

  @override
  void dispose() {
    _controller.dispose(); // Important: dispose ChangeNotifier
    _textController.dispose();
    super.dispose();
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('ChangeNotifier Pattern'),
        actions: [
          // Listen to specific changes using AnimatedBuilder or Consumer pattern
          AnimatedBuilder(
            animation: _controller,
            builder: (context, child) {
              return Chip(
                label: Text('${_controller.activeCount} left'),
              );
            },
          ),
          SizedBox(width: 8),
        ],
      ),
      body: Column(
        children: [
          // Input field
          Padding(
            padding: EdgeInsets.all(16),
            child: Row(
              children: [
                Expanded(
                  child: TextField(
                    controller: _textController,
                    decoration: InputDecoration(
                      hintText: 'What needs to be done?',
                      border: OutlineInputBorder(),
                    ),
                    onSubmitted: (value) {
                      _controller.addTodo(value);
                      _textController.clear();
                    },
                  ),
                ),
                SizedBox(width: 8),
                ElevatedButton(
                  onPressed: () {
                    _controller.addTodo(_textController.text);
                    _textController.clear();
                  },
                  child: Text('Add'),
                ),
              ],
            ),
          ),

          // Filter buttons
          AnimatedBuilder(
            animation: _controller,
            builder: (context, child) {
              return Row(
                mainAxisAlignment: MainAxisAlignment.center,
                children: [
                  _FilterButton(
                    title: 'All',
                    isActive: _controller.filter == 'all',
                    onTap: () => _controller.setFilter('all'),
                  ),
                  _FilterButton(
                    title: 'Active',
                    isActive: _controller.filter == 'active',
                    onTap: () => _controller.setFilter('active'),
                  ),
                  _FilterButton(
                    title: 'Completed',
                    isActive: _controller.filter == 'completed',
                    onTap: () => _controller.setFilter('completed'),
                  ),
                ],
              );
            },
          ),

          // Todo list
          Expanded(
            child: AnimatedBuilder(
              animation: _controller,
              builder: (context, child) {
                if (_controller.todos.isEmpty) {
                  return Center(child: Text('No todos yet!'));
                }
                
                return ListView.builder(
                  itemCount: _controller.todos.length,
                  itemBuilder: (context, index) {
                    final todo = _controller.todos[index];
                    return ListTile(
                      leading: Checkbox(
                        value: todo.isCompleted,
                        onChanged: (_) => _controller.toggleTodo(todo.id),
                      ),
                      title: Text(
                        todo.title,
                        style: TextStyle(
                          decoration: todo.isCompleted
                              ? TextDecoration.lineThrough
                              : null,
                        ),
                      ),
                      trailing: IconButton(
                        icon: Icon(Icons.delete, color: Colors.red),
                        onPressed: () => _controller.removeTodo(todo.id),
                      ),
                    );
                  },
                );
              },
            ),
          ),

          // Clear completed button
          AnimatedBuilder(
            animation: _controller,
            builder: (context, child) {
              if (_controller.completedCount == 0) return SizedBox.shrink();
              
              return Padding(
                padding: EdgeInsets.all(16),
                child: TextButton(
                  onPressed: _controller.clearCompleted,
                  child: Text('Clear completed (${_controller.completedCount})'),
                ),
              );
            },
          ),
        ],
      ),
    );
  }
}

class _FilterButton extends StatelessWidget {
  final String title;
  final bool isActive;
  final VoidCallback onTap;

  const _FilterButton({
    Key? key,
    required this.title,
    required this.isActive,
    required this.onTap,
  }) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Padding(
      padding: EdgeInsets.symmetric(horizontal: 4),
      child: TextButton(
        style: TextButton.styleFrom(
          backgroundColor: isActive ? Colors.blue : null,
          foregroundColor: isActive ? Colors.white : null,
        ),
        onPressed: onTap,
        child: Text(title),
      ),
    );
  }
}
```

**Explanation:**

- **`ChangeNotifier`**: A class that provides change notification to its listeners. It's the foundation for Provider and Riverpod.

- **`notifyListeners()`**: Call this method after changing state. It notifies all registered listeners (typically widgets) that they should rebuild.

- **`AnimatedBuilder`**: While designed for animations, it's a convenient way to listen to any Listenable (including ChangeNotifier) and rebuild only specific parts of the UI.

- **Architecture pattern**:
  1. Create a controller class extending ChangeNotifier
  2. Keep state private, expose public getters
  3. Provide methods to modify state
  4. Call notifyListeners() after state changes
  5. Listen using AnimatedBuilder or similar widgets

- **Advantages over setState**:
  - Business logic separated from UI
  - Testable (can test controller without widget tree)
  - Granular rebuilds (only listening widgets rebuild)
  - Can be shared across multiple screens

- **Disadvantages**:
  - Manual disposal required
  - No dependency injection (must pass instances manually)
  - Can become verbose with deep widget trees

---

## **11.3 Provider Pattern**

Provider is the most widely used state management solution in Flutter. It wraps InheritedWidget to provide dependency injection and reactive state management with minimal boilerplate.

### **Provider Setup and Basic Usage**

```dart
import 'package:flutter/material.dart';
import 'package:provider/provider.dart'; // Add to pubspec.yaml

void main() {
  runapp(ProviderDemo());
}

// State class using ChangeNotifier
class AuthState extends ChangeNotifier {
  bool _isAuthenticated = false;
  User? _user;

  bool get isAuthenticated => _isAuthenticated;
  User? get user => _user;

  void login(String email, String password) async {
    // Simulate API call
    await Future.delayed(Duration(seconds: 1));
    
    _user = User(email: email, name: 'John Doe');
    _isAuthenticated = true;
    notifyListeners();
  }

  void logout() {
    _user = null;
    _isAuthenticated = false;
    notifyListeners();
  }
}

class User {
  final String email;
  final String name;

  User({required this.email, required this.name});
}

// MultiProvider allows providing multiple dependencies
class ProviderDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MultiProvider(
      providers: [
        // ChangeNotifierProvider creates and disposes ChangeNotifier automatically
        ChangeNotifierProvider(create: (_) => AuthState()),
        
        // Provider for immutable objects or services
        Provider(create: (_) => ApiService()),
        
        // ProxyProvider for dependencies that depend on other providers
        // ProxyProvider<AuthState, UserService>(
        //   update: (_, auth, __) => UserService(auth),
        // ),
      ],
      child: MaterialApp(
        home: AuthWrapper(),
      ),
    );
  }
}

class AuthWrapper extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    // Watch listens to changes and rebuilds when notified
    final auth = context.watch<AuthState>();
    
    return Scaffold(
      body: auth.isAuthenticated ? HomeScreen() : LoginScreen(),
    );
  }
}

class LoginScreen extends StatelessWidget {
  final TextEditingController _emailController = TextEditingController();
  final TextEditingController _passwordController = TextEditingController();

  @override
  Widget build(BuildContext context) {
    // Read gets the value once without listening
    // Use for one-time actions like button presses
    final auth = context.read<AuthState>();
    
    return Scaffold(
      appBar: AppBar(title: Text('Login')),
      body: Padding(
        padding: EdgeInsets.all(16),
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            TextField(
              controller: _emailController,
              decoration: InputDecoration(labelText: 'Email'),
            ),
            TextField(
              controller: _passwordController,
              decoration: InputDecoration(labelText: 'Password'),
              obscureText: true,
            ),
            SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                auth.login(
                  _emailController.text,
                  _passwordController.text,
                );
              },
              child: Text('Login'),
            ),
          ],
        ),
      ),
    );
  }
}

class HomeScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    // Select allows listening to specific parts of state for efficiency
    // Only rebuilds when user name changes, not when other auth state changes
    final userName = context.select((AuthState auth) => auth.user?.name);
    
    return Scaffold(
      appBar: AppBar(
        title: Text('Welcome $userName'),
        actions: [
          IconButton(
            icon: Icon(Icons.logout),
            onPressed: () => context.read<AuthState>().logout(),
          ),
        ],
      ),
      body: Center(child: Text('You are logged in!')),
    );
  }
}

// Dummy service class
class ApiService {
  void fetchData() {}
}
```

**Explanation:**

- **`ChangeNotifierProvider`**: Creates and manages a ChangeNotifier. Automatically calls dispose() when the widget is removed from the tree.

- **`context.watch<T>()`**: Listens to the provider and rebuilds the widget whenever notifyListeners() is called. Use in build methods.

- **`context.read<T>()`**: Gets the provider value once without listening. Use in callbacks (onPressed, initState, etc.) where you don't need to rebuild.

- **`context.select<T, R>()`**: Listens to only a specific part of the state. More efficient than watch() when you only need a subset of data.

- **`MultiProvider`**: Combines multiple providers. Cleaner than nesting Provider widgets.

- **Provider types**:
  - `Provider`: For immutable objects or services
  - `ChangeNotifierProvider`: For ChangeNotifiers
  - `ValueListenableProvider`: For ValueNotifiers
  - `StreamProvider`: For Streams
  - `FutureProvider`: For Futures
  - `ProxyProvider`: For dependencies that depend on other providers

### **Provider with Consumer and Selector**

```dart
import 'package:flutter/material.dart';
import 'package:provider/provider.dart';

class ShoppingCart extends ChangeNotifier {
  final List<CartItem> _items = [];

  List<CartItem> get items => List.unmodifiable(_items);
  
  double get totalPrice => _items.fold(
    0, 
    (sum, item) => sum + (item.price * item.quantity),
  );
  
  int get itemCount => _items.fold(0, (sum, item) => sum + item.quantity);

  void addItem(Product product) {
    final existingIndex = _items.indexWhere((item) => item.product.id == product.id);
    
    if (existingIndex >= 0) {
      _items[existingIndex] = _items[existingIndex].copyWith(
        quantity: _items[existingIndex].quantity + 1,
      );
    } else {
      _items.add(CartItem(product: product, quantity: 1));
    }
    notifyListeners();
  }

  void removeItem(String productId) {
    _items.removeWhere((item) => item.product.id == productId);
    notifyListeners();
  }

  void updateQuantity(String productId, int quantity) {
    if (quantity <= 0) {
      removeItem(productId);
      return;
    }
    
    final index = _items.indexWhere((item) => item.product.id == productId);
    if (index >= 0) {
      _items[index] = _items[index].copyWith(quantity: quantity);
      notifyListeners();
    }
  }
}

class Product {
  final String id;
  final String name;
  final double price;
  final String image;

  Product({
    required this.id,
    required this.name,
    required this.price,
    required this.image,
  });
}

class CartItem {
  final Product product;
  final int quantity;

  CartItem({required this.product, required this.quantity});

  double get total => product.price * quantity;

  CartItem copyWith({Product? product, int? quantity}) {
    return CartItem(
      product: product ?? this.product,
      quantity: quantity ?? this.quantity,
    );
  }
}

class ProviderAdvancedDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return ChangeNotifierProvider(
      create: (_) => ShoppingCart(),
      child: MaterialApp(
        home: ProductCatalog(),
      ),
    );
  }
}

class ProductCatalog extends StatelessWidget {
  final List<Product> products = [
    Product(id: '1', name: 'Laptop', price: 999.99, image: '💻'),
    Product(id: '2', name: 'Mouse', price: 29.99, image: '🖱️'),
    Product(id: '3', name: 'Keyboard', price: 79.99, image: '⌨️'),
    Product(id: '4', name: 'Monitor', price: 299.99, image: '🖥️'),
  ];

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Shop'),
        actions: [
          // Consumer rebuilds only this widget when cart changes
          Consumer<ShoppingCart>(
            builder: (context, cart, child) {
              return Badge(
                label: Text('${cart.itemCount}'),
                child: IconButton(
                  icon: Icon(Icons.shopping_cart),
                  onPressed: () {
                    Navigator.push(
                      context,
                      MaterialPageRoute(builder: (_) => CartScreen()),
                    );
                  },
                ),
              );
            },
          ),
        ],
      ),
      body: ListView.builder(
        itemCount: products.length,
        itemBuilder: (context, index) {
          final product = products[index];
          return ProductCard(product: product);
        },
      ),
    );
  }
}

class ProductCard extends StatelessWidget {
  final Product product;

  const ProductCard({Key? key, required this.product}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Card(
      margin: EdgeInsets.all(8),
      child: Padding(
        padding: EdgeInsets.all(16),
        child: Row(
          children: [
            Text(product.image, style: TextStyle(fontSize: 40)),
            SizedBox(width: 16),
            Expanded(
              child: Column(
                crossAxisAlignment: CrossAxisAlignment.start,
                children: [
                  Text(
                    product.name,
                    style: TextStyle(fontSize: 18, fontWeight: FontWeight.bold),
                  ),
                  Text('\$${product.price.toStringAsFixed(2)}'),
                ],
              ),
            ),
            ElevatedButton(
              onPressed: () {
                context.read<ShoppingCart>().addItem(product);
                ScaffoldMessenger.of(context).showSnackBar(
                  SnackBar(
                    content: Text('${product.name} added to cart'),
                    duration: Duration(seconds: 1),
                  ),
                );
              },
              child: Text('Add'),
            ),
          ],
        ),
      ),
    );
  }
}

class CartScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Shopping Cart')),
      body: Column(
        children: [
          Expanded(
            // Selector allows fine-grained control over rebuilds
            // Only rebuilds when the list of items changes, not when quantities change
            child: Selector<ShoppingCart, List<CartItem>>(
              selector: (_, cart) => cart.items,
              builder: (context, items, child) {
                if (items.isEmpty) {
                  return Center(child: Text('Your cart is empty'));
                }
                
                return ListView.builder(
                  itemCount: items.length,
                  itemBuilder: (context, index) {
                    final item = items[index];
                    return CartItemTile(item: item);
                  },
                );
              },
            ),
          ),
          
          // Consumer for the total price section
          Consumer<ShoppingCart>(
            builder: (context, cart, child) {
              return Container(
                padding: EdgeInsets.all(16),
                decoration: BoxDecoration(
                  color: Colors.grey[200],
                  border: Border(top: BorderSide(color: Colors.grey)),
                ),
                child: SafeArea(
                  child: Column(
                    mainAxisSize: MainAxisSize.min,
                    children: [
                      Row(
                        mainAxisAlignment: MainAxisAlignment.spaceBetween,
                        children: [
                          Text(
                            'Total:',
                            style: TextStyle(
                              fontSize: 20,
                              fontWeight: FontWeight.bold,
                            ),
                          ),
                          Text(
                            '\$${cart.totalPrice.toStringAsFixed(2)}',
                            style: TextStyle(
                              fontSize: 20,
                              fontWeight: FontWeight.bold,
                              color: Colors.green,
                            ),
                          ),
                        ],
                      ),
                      SizedBox(height: 16),
                      SizedBox(
                        width: double.infinity,
                        child: ElevatedButton(
                          onPressed: cart.items.isEmpty ? null : () {},
                          child: Text('Checkout'),
                        ),
                      ),
                    ],
                  ),
                ),
              );
            },
          ),
        ],
      ),
    );
  }
}

class CartItemTile extends StatelessWidget {
  final CartItem item;

  const CartItemTile({Key? key, required this.item}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return ListTile(
      leading: Text(item.product.image, style: TextStyle(fontSize: 30)),
      title: Text(item.product.name),
      subtitle: Text('\$${item.product.price} each'),
      trailing: Row(
        mainAxisSize: MainAxisSize.min,
        children: [
          IconButton(
            icon: Icon(Icons.remove),
            onPressed: () {
              context.read<ShoppingCart>().updateQuantity(
                item.product.id,
                item.quantity - 1,
              );
            },
          ),
          Text('${item.quantity}'),
          IconButton(
            icon: Icon(Icons.add),
            onPressed: () {
              context.read<ShoppingCart>().updateQuantity(
                item.product.id,
                item.quantity + 1,
              );
            },
          ),
          IconButton(
            icon: Icon(Icons.delete, color: Colors.red),
            onPressed: () {
              context.read<ShoppingCart>().removeItem(item.product.id);
            },
          ),
        ],
      ),
    );
  }
}

// Badge widget for cart icon
class Badge extends StatelessWidget {
  final Widget child;
  final Widget label;

  const Badge({Key? key, required this.child, required this.label}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return Stack(
      alignment: Alignment.center,
      children: [
        child,
        Positioned(
          top: 0,
          right: 0,
          child: Container(
            padding: EdgeInsets.all(2),
            decoration: BoxDecoration(
              color: Colors.red,
              borderRadius: BorderRadius.circular(10),
            ),
            constraints: BoxConstraints(minWidth: 16, minHeight: 16),
            child: Center(
              child: DefaultTextStyle(
                style: TextStyle(color: Colors.white, fontSize: 10),
                child: label,
              ),
            ),
          ),
        ),
      ],
    );
  }
}
```

**Explanation:**

- **`Consumer<T>`**: A widget that listens to a provider and rebuilds when it changes. Useful when you need to listen in a specific part of the widget tree without rebuilding the entire parent.

- **`Selector<T, R>`**: More efficient than Consumer. Allows you to select a specific part of the state to listen to. Only rebuilds when the selected value changes.
  - `selector`: Function that extracts the value to listen to
  - `builder`: Receives the selected value
  - `shouldRebuild`: Optional custom comparison logic

- **Performance optimization**:
  - Use `context.select()` for simple selections in build methods
  - Use `Selector` for complex derived data or widgets deep in the tree
  - Use `Consumer` when you need the entire object and want to localize rebuilds

- **When to use what**:
  - `context.watch()`: When the whole widget needs to rebuild on any change
  - `context.select()`: When only specific data is needed
  - `Consumer`: To optimize rebuilds in large widget trees
  - `Selector`: For expensive calculations or when listening to specific properties

---

## **11.4 Riverpod (Next-Gen State Management)**

Riverpod is a rewrite of Provider that fixes its limitations. It is compile-safe, testing-friendly, and doesn't depend on Flutter's widget tree.

### **Riverpod Basics**

```dart
import 'package:flutter/material.dart';
import 'package:flutter_riverpod/flutter_riverpod.dart'; // Add to pubspec.yaml

void main() {
  // Wrap app with ProviderScope
  runApp(ProviderScope(child: RiverpodDemo()));
}

// Providers are global variables that define how to create state
// They are immutable and can be accessed from anywhere

// Provider: For immutable values or services
final apiServiceProvider = Provider<ApiService>((ref) {
  return ApiService();
});

// StateProvider: For simple state that can change (primitives, simple objects)
final counterProvider = StateProvider<int>((ref) => 0);

// StateNotifierProvider: For complex state with logic
final todoProvider = StateNotifierProvider<TodoNotifier, List<Todo>>((ref) {
  return TodoNotifier();
});

// FutureProvider: For async operations
final userProvider = FutureProvider<User>((ref) async {
  final api = ref.read(apiServiceProvider);
  return api.fetchUser();
});

// StreamProvider: For Streams
final clockProvider = StreamProvider<DateTime>((ref) {
  return Stream.periodic(Duration(seconds: 1), (_) => DateTime.now());
});

class RiverpodDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      home: HomePage(),
    );
  }
}

class HomePage extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    // Watch the counter provider
    final count = ref.watch(counterProvider);
    
    return Scaffold(
      appBar: AppBar(title: Text('Riverpod')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Count: $count', style: TextStyle(fontSize: 24)),
            SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // Read to modify state
                ref.read(counterProvider.notifier).state++;
              },
              child: Text('Increment'),
            ),
            SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                Navigator.push(
                  context,
                  MaterialPageRoute(builder: (_) => TodoPage()),
                );
              },
              child: Text('Go to Todos'),
            ),
          ],
        ),
      ),
    );
  }
}

// StateNotifier for complex state
class TodoNotifier extends StateNotifier<List<Todo>> {
  TodoNotifier() : super([]);

  void add(String title) {
    state = [
      ...state,
      Todo(id: DateTime.now().toString(), title: title),
    ];
  }

  void toggle(String id) {
    state = state.map((todo) {
      if (todo.id == id) {
        return todo.copyWith(isCompleted: !todo.isCompleted);
      }
      return todo;
    }).toList();
  }

  void remove(String id) {
    state = state.where((todo) => todo.id != id).toList();
  }
}

class Todo {
  final String id;
  final String title;
  final bool isCompleted;

  Todo({
    required this.id,
    required this.title,
    this.isCompleted = false,
  });

  Todo copyWith({String? id, String? title, bool? isCompleted}) {
    return Todo(
      id: id ?? this.id,
      title: title ?? this.title,
      isCompleted: isCompleted ?? this.isCompleted,
    );
  }
}

class TodoPage extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    // Watch todo list
    final todos = ref.watch(todoProvider);
    
    return Scaffold(
      appBar: AppBar(title: Text('Todos (${todos.length})')),
      body: ListView.builder(
        itemCount: todos.length,
        itemBuilder: (context, index) {
          final todo = todos[index];
          return ListTile(
            leading: Checkbox(
              value: todo.isCompleted,
              onChanged: (_) {
                ref.read(todoProvider.notifier).toggle(todo.id);
              },
            ),
            title: Text(
              todo.title,
              style: TextStyle(
                decoration: todo.isCompleted ? TextDecoration.lineThrough : null,
              ),
            ),
            trailing: IconButton(
              icon: Icon(Icons.delete),
              onPressed: () {
                ref.read(todoProvider.notifier).remove(todo.id);
              },
            ),
          );
        },
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: () {
          _showAddDialog(context, ref);
        },
        child: Icon(Icons.add),
      ),
    );
  }

  void _showAddDialog(BuildContext context, WidgetRef ref) {
    final controller = TextEditingController();
    
    showDialog(
      context: context,
      builder: (context) => AlertDialog(
        title: Text('New Todo'),
        content: TextField(controller: controller),
        actions: [
          TextButton(
            onPressed: () => Navigator.pop(context),
            child: Text('Cancel'),
          ),
          TextButton(
            onPressed: () {
              if (controller.text.isNotEmpty) {
                ref.read(todoProvider.notifier).add(controller.text);
              }
              Navigator.pop(context);
            },
            child: Text('Add'),
          ),
        ],
      ),
    );
  }
}

// Dummy classes
class ApiService {
  Future<User> fetchUser() async {
    await Future.delayed(Duration(seconds: 1));
    return User(name: 'John');
  }
}

class User {
  final String name;
  User({required this.name});
}
```

**Explanation:**

- **`ProviderScope`**: The root widget that stores the state of all providers. Must wrap your app.

- **Provider types**:
  - `Provider`: Immutable values, services, computed values
  - `StateProvider`: Simple mutable state (primitives, simple objects)
  - `StateNotifierProvider`: Complex state with business logic (similar to ChangeNotifier)
  - `FutureProvider`: Async values (API calls)
  - `StreamProvider`: Stream values (real-time updates)
  - `ScopedProvider`: Override providers for specific subtrees

- **WidgetRef**: Passed to ConsumerWidget/ConsumerStatefulWidget. Provides:
  - `ref.watch(provider)`: Listen to provider, rebuild on change
  - `ref.read(provider)`: Get value once without listening
  - `ref.listen(provider, callback)`: Listen and execute side effects

- **StateProvider**:
  - Access state: `ref.watch(counterProvider)`
  - Modify state: `ref.read(counterProvider.notifier).state = newValue`

- **StateNotifierProvider**:
  - Access state: `ref.watch(todoProvider)` (returns the state, not the notifier)
  - Call methods: `ref.read(todoProvider.notifier).add('title')`

- **Advantages over Provider**:
  - Compile-safe (no runtime exceptions for missing providers)
  - Can be accessed outside widget tree (in tests, other classes)
  - Automatic caching and disposal
  - Family modifiers for parameterized providers
  - Better support for async/streams

### **Riverpod Advanced Features**

```dart
import 'package:flutter/material.dart';
import 'package:flutter_riverpod/flutter_riverpod.dart';

// AutoDispose: Automatically dispose when no longer used
final autoDisposeProvider = FutureProvider.autoDispose<String>((ref) async {
  // This provider will be disposed when the widget is removed
  // and recreated when the widget is added again
  
  // Keep alive for 5 minutes to prevent flickering
  ref.keepAlive();
  
  await Future.delayed(Duration(seconds: 1));
  return 'Fetched Data';
});

// Family: Create multiple instances with parameters
final userProvider = FutureProvider.family<User, String>((ref, userId) async {
  final api = ref.read(apiServiceProvider);
  return api.fetchUser(userId);
});

// ScopedProvider with overrides
final themeProvider = StateProvider<ThemeMode>((ref) => ThemeMode.light);

class RiverpodAdvancedDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return ProviderScope(
      child: ConsumerWidget(
        builder: (context, ref, child) {
          final themeMode = ref.watch(themeProvider);
          
          return MaterialApp(
            themeMode: themeMode,
            theme: ThemeData.light(),
            darkTheme: ThemeData.dark(),
            home: AdvancedExamplesPage(),
          );
        },
      ),
    );
  }
}

class AdvancedExamplesPage extends ConsumerWidget {
  @override
  Widget build(BuildContext context, WidgetRef ref) {
    return Scaffold(
      appBar: AppBar(title: Text('Riverpod Advanced')),
      body: ListView(
        children: [
          // AutoDispose example
          _buildSection('AutoDispose Provider'),
          Consumer(
            builder: (context, ref, child) {
              final asyncValue = ref.watch(autoDisposeProvider);
              
              return asyncValue.when(
                data: (data) => ListTile(
                  title: Text(data),
                  trailing: Icon(Icons.check),
                ),
                loading: () => ListTile(
                  title: LinearProgressIndicator(),
                ),
                error: (err, stack) => ListTile(
                  title: Text('Error: $err'),
                  trailing: Icon(Icons.error),
                ),
              );
            },
          ),
          
          // Family example
          _buildSection('Family Provider'),
          ...List.generate(3, (index) {
            return UserTile(userId: 'user_$index');
          }),
          
          // State persistence example
          _buildSection('Theme Switcher'),
          ListTile(
            title: Text('Toggle Theme'),
            trailing: Switch(
              value: ref.watch(themeProvider) == ThemeMode.dark,
              onChanged: (value) {
                ref.read(themeProvider.notifier).state = 
                    value ? ThemeMode.dark : ThemeMode.light;
              },
            ),
          ),
        ],
      ),
    );
  }
  
  Widget _buildSection(String title) {
    return Padding(
      padding: EdgeInsets.fromLTRB(16, 16, 16, 8),
      child: Text(
        title,
        style: TextStyle(fontSize: 18, fontWeight: FontWeight.bold, color: Colors.blue),
      ),
    );
  }
}

class UserTile extends ConsumerWidget {
  final String userId;

  const UserTile({Key? key, required this.userId}) : super(key: key);

  @override
  Widget build(BuildContext context, WidgetRef ref) {
    // Using family provider with parameter
    final userAsync = ref.watch(userProvider(userId));
    
    return userAsync.when(
      data: (user) => ListTile(
        title: Text(user.name),
        subtitle: Text('ID: $userId'),
      ),
      loading: () => ListTile(title: Text('Loading $userId...')),
      error: (err, _) => ListTile(title: Text('Error loading $userId')),
    );
  }
}

// Dummy classes
class ApiService {
  Future<User> fetchUser(String id) async {
    await Future.delayed(Duration(milliseconds: 500));
    return User(name: 'User ${id.split('_').last}');
  }
}

class User {
  final String name;
  User({required this.name});
}
```

**Explanation:**

- **`.autoDispose`**: Automatically disposes the provider state when no widgets are listening. Prevents memory leaks.

- **`.family`**: Creates a provider that accepts parameters. Each unique parameter creates a separate state instance.

- **`ref.keepAlive()`**: Prevents autoDispose from disposing the state immediately. Useful for caching expensive operations.

- **`.when()`**: Convenience method on AsyncValue (returned by FutureProvider/StreamProvider):
  - `data`: Widget to show when data is available
  - `loading`: Widget to show while loading
  - `error`: Widget to show on error

- **Scoped overrides**: Use `ProviderScope(overrides: [...])` to override providers for specific widget subtrees (useful for testing or theming).

---

## **11.5 BLoC Pattern (Business Logic Component)**

BLoC is an architectural pattern that separates business logic from UI using Streams and Events. It's excellent for complex applications with strict separation of concerns requirements.

### **BLoC Implementation**

```dart
import 'package:flutter/material.dart';
import 'package:flutter_bloc/flutter_bloc.dart'; // Add flutter_bloc to pubspec.yaml

void main() {
  runapp(BlocDemo());
}

// Events: Input to the BLoC (user actions, system events)
abstract class CounterEvent {}

class CounterIncrementPressed extends CounterEvent {}

class CounterDecrementPressed extends CounterEvent {}

class CounterResetPressed extends CounterEvent {}

// State: Output from the BLoC (UI state)
class CounterState {
  final int count;
  final bool isLoading;

  const CounterState({
    this.count = 0,
    this.isLoading = false,
  });

  CounterState copyWith({int? count, bool? isLoading}) {
    return CounterState(
      count: count ?? this.count,
      isLoading: isLoading ?? this.isLoading,
    );
  }
}

// BLoC: Business Logic Component
// Transforms events into states
class CounterBloc extends Bloc<CounterEvent, CounterState> {
  CounterBloc() : super(const CounterState()) {
    // Register event handlers
    on<CounterIncrementPressed>(_onIncrement);
    on<CounterDecrementPressed>(_onDecrement);
    on<CounterResetPressed>(_onReset);
  }

  void _onIncrement(CounterIncrementPressed event, Emitter<CounterState> emit) {
    emit(state.copyWith(isLoading: true));
    
    // Simulate async operation
    await Future.delayed(Duration(milliseconds: 500));
    
    emit(state.copyWith(
      count: state.count + 1,
      isLoading: false,
    ));
  }

  void _onDecrement(CounterDecrementPressed event, Emitter<CounterState> emit) {
    emit(state.copyWith(count: state.count - 1));
  }

  void _onReset(CounterResetPressed event, Emitter<CounterState> emit) {
    emit(const CounterState());
  }
}

class BlocDemo extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      home: BlocProvider(
        create: (_) => CounterBloc(),
        child: CounterPage(),
      ),
    );
  }
}

class CounterPage extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('BLoC Pattern')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            // BlocBuilder rebuilds when state changes
            BlocBuilder<CounterBloc, CounterState>(
              builder: (context, state) {
                if (state.isLoading) {
                  return CircularProgressIndicator();
                }
                return Text(
                  '${state.count}',
                  style: TextStyle(fontSize: 72, fontWeight: FontWeight.bold),
                );
              },
            ),
            SizedBox(height: 20),
            Row(
              mainAxisAlignment: MainAxisAlignment.center,
              children: [
                ElevatedButton(
                  onPressed: () {
                    // Add event to BLoC
                    context.read<CounterBloc>().add(CounterDecrementPressed());
                  },
                  child: Icon(Icons.remove),
                ),
                SizedBox(width: 16),
                ElevatedButton(
                  onPressed: () {
                    context.read<CounterBloc>().add(CounterResetPressed());
                  },
                  child: Icon(Icons.refresh),
                ),
                SizedBox(width: 16),
                ElevatedButton(
                  onPressed: () {
                    context.read<CounterBloc>().add(CounterIncrementPressed());
                  },
                  child: Icon(Icons.add),
                ),
              ],
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **Event**: An object representing an action or intention (user clicked button, data received, etc.). Events flow INTO the BLoC.

- **State**: An object representing the current UI state. States flow OUT OF the BLoC.

- **BLoC**: The business logic component that receives events and emits new states. It uses `on<EventType>()` to register handlers.

- **Emitter**: Function passed to event handlers that allows emitting new states. States must be immutable.

- **BlocProvider**: Dependency injection widget that creates and provides the BLoC to the widget tree.

- **BlocBuilder**: Widget that listens to state changes and rebuilds. Similar to Consumer in Provider.

- **context.read()**: Gets the BLoC to add events without listening.

- **Unidirectional Data Flow**:
  1. User interaction triggers Event
  2. BLoC receives Event and runs business logic
  3. BLoC emits new State
  4. UI rebuilds based on new State

### **BLoC with API Integration**

```dart
import 'package:flutter/material.dart';
import 'package:flutter_bloc/flutter_bloc.dart';
import 'package:equatable/equatable.dart'; // For easy equality checks

// Events
abstract class PostEvent {}

class PostFetched extends PostEvent {}

class PostRefreshed extends PostEvent {}

// States
abstract class PostState extends Equatable {
  const PostState();
  
  @override
  List<Object> get props => [];
}

class PostInitial extends PostState {}

class PostLoading extends PostState {}

class PostSuccess extends PostState {
  final List<Post> posts;
  final bool hasReachedMax;

  const PostSuccess({
    required this.posts,
    this.hasReachedMax = false,
  });

  PostSuccess copyWith({List<Post>? posts, bool? hasReachedMax}) {
    return PostSuccess(
      posts: posts ?? this.posts,
      hasReachedMax: hasReachedMax ?? this.hasReachedMax,
    );
  }

  @override
  List<Object> get props => [posts, hasReachedMax];
}

class PostFailure extends PostState {
  final String error;
  
  const PostFailure(this.error);
  
  @override
  List<Object> get props => [error];
}

// BLoC
class PostBloc extends Bloc<PostEvent, PostState> {
  final ApiService api;
  int _page = 1;

  PostBloc({required this.api}) : super(PostInitial()) {
    on<PostFetched>(_onFetched);
    on<PostRefreshed>(_onRefreshed);
  }

  Future<void> _onFetched(PostFetched event, Emitter<PostState> emit) async {
    if (state is PostSuccess && (state as PostSuccess).hasReachedMax) return;

    try {
      if (state is PostInitial) {
        emit(PostLoading());
        final posts = await api.fetchPosts(page: 1);
        emit(PostSuccess(posts: posts, hasReachedMax: posts.isEmpty));
      } else if (state is PostSuccess) {
        final currentState = state as PostSuccess;
        _page++;
        final posts = await api.fetchPosts(page: _page);
        emit(posts.isEmpty
            ? currentState.copyWith(hasReachedMax: true)
            : PostSuccess(
                posts: currentState.posts + posts,
                hasReachedMax: false,
              ));
      }
    } catch (e) {
      emit(PostFailure(e.toString()));
    }
  }

  Future<void> _onRefreshed(PostRefreshed event, Emitter<PostState> emit) async {
    _page = 1;
    emit(PostLoading());
    try {
      final posts = await api.fetchPosts(page: 1);
      emit(PostSuccess(posts: posts, hasReachedMax: posts.isEmpty));
    } catch (e) {
      emit(PostFailure(e.toString()));
    }
  }
}

class Post {
  final int id;
  final String title;
  final String body;

  Post({required this.id, required this.title, required this.body});
}

class ApiService {
  Future<List<Post>> fetchPosts({required int page}) async {
    await Future.delayed(Duration(seconds: 1)); // Simulate network
    
    // Simulate pagination end
    if (page > 3) return [];
    
    return List.generate(10, (index) => Post(
      id: (page - 1) * 10 + index,
      title: 'Post ${(page - 1) * 10 + index}',
      body: 'This is the body of post ${(page - 1) * 10 + index}',
    ));
  }
}

class PostListPage extends StatefulWidget {
  @override
  _PostListPageState createState() => _PostListPageState();
}

class _PostListPageState extends State<PostListPage> {
  final ScrollController _scrollController = ScrollController();

  @override
  void initState() {
    super.initState();
    _scrollController.addListener(_onScroll);
  }

  @override
  void dispose() {
    _scrollController.dispose();
    super.dispose();
  }

  void _onScroll() {
    if (_isBottom) {
      context.read<PostBloc>().add(PostFetched());
    }
  }

  bool get _isBottom {
    if (!_scrollController.hasClients) return false;
    final maxScroll = _scrollController.position.maxScrollExtent;
    final currentScroll = _scrollController.offset;
    return currentScroll >= (maxScroll * 0.9);
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Posts with BLoC')),
      body: BlocBuilder<PostBloc, PostState>(
        builder: (context, state) {
          if (state is PostFailure) {
            return Center(
              child: Text('Error: ${(state as PostFailure).error}'),
            );
          }
          
          if (state is PostSuccess) {
            final posts = (state as PostSuccess).posts;
            final hasReachedMax = (state as PostSuccess).hasReachedMax;
            
            return RefreshIndicator(
              onRefresh: () async {
                context.read<PostBloc>().add(PostRefreshed());
              },
              child: ListView.builder(
                controller: _scrollController,
                itemCount: hasReachedMax ? posts.length : posts.length + 1,
                itemBuilder: (context, index) {
                  if (index >= posts.length) {
                    return BottomLoader();
                  }
                  final post = posts[index];
                  return ListTile(
                    title: Text(post.title),
                    subtitle: Text(post.body, maxLines: 2, overflow: TextOverflow.ellipsis),
                  );
                },
              ),
            );
          }
          
          return Center(child: CircularProgressIndicator());
        },
      ),
    );
  }
}

class BottomLoader extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Container(
      alignment: Alignment.center,
      padding: EdgeInsets.all(16),
      child: CircularProgressIndicator(),
    );
  }
}
```

**Explanation:**

- **Equatable**: Package that simplifies equality comparisons. Essential for BLoC to determine if state actually changed.

- **Pagination Pattern**: Common BLoC pattern for infinite scrolling:
  - Track current page
  - Append new items to existing list
  - Use `hasReachedMax` flag to stop fetching

- **BlocBuilder**: Rebuilds UI based on state type (loading, success, error).

- **Side Effects**: Use `BlocListener` (instead of Builder) for side effects like showing SnackBars or navigation that shouldn't rebuild the UI.

- **Cubit**: Simplified version of BLoC that uses methods instead of events. Good for simple state management without complex event logic.

---

## **Chapter Summary**

In this chapter, we covered state management architectures in Flutter:

### **Key Takeaways:**

1. **State Classification**:
   - Ephemeral (UI) state: Use setState
   - App (shared) state: Use dedicated solutions
   - Page state: Intermediate scope

2. **ValueNotifier/ChangeNotifier**:
   - Built-in Flutter solutions
   - Good for medium complexity
   - Manual disposal required
   - Testable but lacks dependency injection

3. **Provider**:
   - Most popular solution
   - Wraps InheritedWidget
   - ChangeNotifierProvider for state
   - Consumer and Selector for granular rebuilds
   - watch/read/select patterns

4. **Riverpod**:
   - Compile-safe alternative to Provider
   - Global providers, no BuildContext needed
   - AutoDispose for memory management
   - Family for parameterized providers
   - Excellent async support

5. **BLoC**:
   - Strict separation of concerns
   - Event-driven architecture
   - Unidirectional data flow
   - Excellent for complex business logic
   - Steeper learning curve but very scalable

### **When to Choose What**:

- **setState**: Single widget, simple UI updates
- **ChangeNotifier**: Small apps, 2-3 screens sharing state
- **Provider**: Medium apps, team environments, standard architecture
- **Riverpod**: Modern apps, need for safety, complex async flows
- **BLoC**: Large enterprise apps, complex business logic, strict testing requirements

### **Best Practices**:

- Keep business logic out of widgets
- Use immutable state objects
- Select specific state slices to prevent unnecessary rebuilds
- Dispose controllers and notifiers properly
- Test business logic independently of UI
- Choose the simplest solution that meets your needs

### **Next Steps**:

Now that you understand state management, the next chapter will cover **Chapter 12: Navigation & Routing**, including:

- Navigator and Route classes
- Named routes and onGenerateRoute
- Navigation 2.0 (Router) for declarative routing
- Deep linking configuration
- GoRouter for simplified routing

---

**End of Chapter 11**

---

# **Next Chapter: Chapter 12 - Navigation & Routing**

Chapter 12 will explore Flutter's navigation system, from basic imperative navigation to declarative routing with Navigation 2.0 and deep linking for web and mobile platforms.